In [1]:
#!pip install git+https://github.com/huggingface/transformers
!pip install qwen-omni-utils -U
!pip install -U bitsandbytes`
import os
import soundfile as sf
import torch
import numpy as np
import librosa
import csv # Nuovo import per la gestione dei file CSV
import transformers
from transformers import Qwen2AudioForConditionalGeneration, AutoProcessor
# --- ABILITA IL DEBUGGING CUDA SYNCHRONOUS E L'ALLOCATORE DI MEMORIA ESPANDIBILE ---
# Questa configurazione aiuta a prevenire errori di memoria frammentata sulla GPU.
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# --- DEBUG GPU DISPONIBILE ---
if torch.cuda.is_available():
    # Stampa le informazioni su tutte le GPU disponibili rilevate da PyTorch.
    # Questo aiuta a capire quante risorse hardware sono a disposizione.
    for i in range(torch.cuda.device_count()):
        print(f"*** CONFERMA GPU: PyTorch rileva la GPU {i}! Nome: {torch.cuda.get_device_name(i)}, Memoria Totale: {torch.cuda.get_device_properties(i).total_memory / (1024**3):.2f} GiB")
else:
    # Avviso se nessuna GPU è disponibile, il che implica un'esecuzione molto più lenta su CPU.
    print("*** ATTENZIONE: PyTorch NON rileva la GPU. Il modello verrà caricato su CPU (potrebbe fallire per memoria).")

# --- 1. CONFIGURAZIONE GENERALE E CARICAMENTO MODELLO/PROCESSORE ---
print("Caricamento del modello Qwen2-Audio-7B-Instruct...")
# Inizializza l'AutoProcessor per il modello Qwen2-Audio-7B-Instruct.
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-Audio-7B-Instruct")

# Carica il modello Qwen2AudioForConditionalGeneration.
# device_map="auto" tenta di distribuire il modello automaticamente sulle GPU disponibili.
# torch_dtype=torch.float16 carica il modello in precisione FP16 per ridurre il consumo di VRAM,
# cruciale per modelli di grandi dimensioni su GPU con memoria limitata.
model = Qwen2AudioForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-Audio-7B-Instruct",
    device_map="auto",
    torch_dtype=torch.float16
)
print("Modello e processore Qwen2-Audio caricati con successo.")

# Svuota la cache CUDA immediatamente dopo il caricamento del modello.
# Questo è un passo cruciale per liberare qualsiasi memoria temporanea allocata durante il caricamento,
# assicurando la massima VRAM disponibile per l'inferenza.
torch.cuda.empty_cache()
print("Cache CUDA svuotata dopo il caricamento del modello.")


# --- FUNZIONE PER L'INFERENZA SU UN SINGOLO FRAMMENTO AUDIO ---
def run_audio_inference_on_segment(audio_segment: np.ndarray, user_prompt: str, model, processor, segment_samplerate: int, max_new_tokens: int = 256) -> str:
    """
    Esegue l'inferenza di Qwen2-Audio su un singolo segmento audio (array numpy) con un dato prompt.

    Args:
        audio_segment (np.ndarray): L'array numpy del segmento audio.
        user_prompt (str): Il prompt testuale specifico per l'utente per questo segmento.
        model: L'istanza del modello Qwen2AudioForConditionalGeneration già caricata.
        processor: L'istanza dell'AutoProcessor per Qwen2-Audio già caricata.
        segment_samplerate (int): Il sample rate del segmento audio.
        max_new_tokens (int): Numero massimo di nuovi token che il modello dovrebbe generare come risposta.

    Returns:
        str: La risposta testuale generata dal modello per il segmento.
    """
    # Stampa la lunghezza del segmento audio in secondi per monitoraggio.
    print(f"\n--- Inizio inferenza su segmento audio (lunghezza: {len(audio_segment)/segment_samplerate:.2f}s) ---")

    # Qwen2-Audio si aspetta un sample rate specifico (generalmente 16000 Hz).
    # Questa sezione ricasella l'audio se il sample rate originale non corrisponde,
    # garantendo la compatibilità con il modello.
    target_sr = 16000
    if segment_samplerate != target_sr:
        print(f"Ricasellamento del segmento audio da {segment_samplerate}Hz a {target_sr}Hz.")
        audio_for_processor = librosa.resample(y=audio_segment, orig_sr=segment_samplerate, target_sr=target_sr)
    else:
        audio_for_processor = audio_segment
    final_samplerate = target_sr # Il sample rate effettivo dopo il ricasellamento.

    # --- Costruzione della conversazione per il modello ---
    # Il prompt di sistema definisce il ruolo del modello e il formato di output desiderato.
    # Non include più la richiesta di valori numerici per le emozioni.
    system_prompt = """Sei un analista musicale altamente specializzato e descrittivo.
Il tuo compito è analizzare il frammento audio fornito e fornire una descrizione concisa ma ricca di dettagli, coprendo gli aspetti richiesti.
La tua risposta deve essere in italiano.
Devi includere:
1.  **Descrizione Dettagliata:** Strumenti, velocità/ritmo, timbro, emozioni suscitate, e un contesto immaginato. (max 4-6 righe)
"""

    conversation = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": [
                {"type": "audio", "audio": audio_for_processor}, # Passa l'array NumPy dell'audio.
                {"type": "text", "text": user_prompt}, # user_prompt è ora il prompt specifico per il frammento.
            ],
        },
    ]

    print("Preparazione degli input per il modello Qwen2-Audio...")
    # Applica il template di chat e tokenizza il testo e l'audio.
    # `padding=True` assicura che tutti gli input siano della stessa lunghezza per il batch processing.
    text_input_formatted = processor.apply_chat_template(conversation, add_generation_prompt=True, tokenize=False)
    
    # La lista degli array audio è necessaria per il processore.
    audios_np_arrays = [audio_for_processor]

    inputs = processor(text=text_input_formatted, audios=audios_np_arrays, sampling_rate=final_samplerate, return_tensors="pt", padding=True)

    # Sposta i tensori di input sul dispositivo del modello (GPU o CPU).
    inputs.input_ids = inputs.input_ids.to(model.device)
    if "input_features" in inputs:
        inputs["input_features"] = inputs["input_features"].to(model.device)
    # Alcuni modelli potrebbero usare 'pixel_values' per le feature audio/multimodali.
    elif "pixel_values" in inputs:
        inputs["pixel_values"] = inputs["pixel_values"].to(model.device)
    else:
        print("Avviso: Nessun 'input_features' o 'pixel_values' trovato negli input. Controlla la documentazione del modello.")

    # Stampa la lunghezza dei token di input per debugging.
    print(f"Lunghezza degli input_ids dopo tokenizzazione: {inputs.input_ids.size(1)}")
    print("Input pronti per la generazione.")

    # Svuota la cache CUDA prima della generazione per massimizzare la memoria disponibile.
    # Questo è fondamentale per prevenire OutOfMemoryError tra le chiamate di inferenza.
    torch.cuda.empty_cache()

    print("Generazione della risposta del modello... Potrebbe richiedere tempo.")
    # Genera la risposta testuale dal modello.
    # `max_new_tokens` controlla la lunghezza massima dell'output generato.
    generate_ids = model.generate(**inputs, max_new_tokens=max_new_tokens)

    # Estrai solo i token generati dal modello (escludendo i token di input).
    generated_ids_only = generate_ids[:, inputs.input_ids.size(1):]
    # Decodifica i token generati in testo leggibile.
    response_text = processor.batch_decode(generated_ids_only, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

    print("\n--- TESTO GENERATO DAL MODELLO ---\n")
    print(response_text)
    print(f"--- Fine inferenza su segmento audio ---\n")

    return response_text

# --- SEZIONE PRINCIPALE: SUDDIVISIONE E ANALISI DI PIÙ AUDIO DA UNA CARTELLA ---

# Imposta il percorso della cartella che contiene i tuoi file audio.
# ASSICURATI CHE QUESTO PERCORSO SIA CORRETTO PER IL TUO AMBIENTE KAGGLE/COLAB.
audio_directory_path = "/kaggle/input/le-musiche/PiccoloSubSetAudio/"

# Verifica che la cartella esista.
if not os.path.exists(audio_directory_path):
    raise FileNotFoundError(f"La cartella audio non trovata: {audio_directory_path}. Assicurati di aver caricato la cartella come Dataset e di aver specificato il percorso corretto.")

# Nome del file CSV in cui verranno salvati i risultati.
csv_output_filename = "analisi_audio_per_canzone.csv"

# Apertura del file CSV in modalità scrittura.
# 'newline=''' evita righe vuote extra.
# 'encoding='utf-8''' per supportare caratteri speciali.
with open(csv_output_filename, 'w', newline='', encoding='utf-8') as csvfile:
    # Definisce le intestazioni delle colonne del CSV.
    fieldnames = ['Nome Canzone']
    for i in range(1, 6): # Per 5 frammenti: 'Descr Frammento 1', 'Descr Frammento 2', ...
        fieldnames.append(f'Descr Frammento {i}')
    fieldnames.append('Descrizione Totale') # Aggiunge la colonna per la descrizione complessiva.
    
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader() # Scrive la riga di intestazione nel CSV.

    print(f"\nI risultati dell'analisi verranno salvati in: {csv_output_filename}")

    # Impostazione fissa del numero di frammenti per ogni audio.
    N_FIXED_SEGMENTS = 5 # Vogliamo esattamente 5 frammenti per ogni file audio.

    # Itera su tutti i file presenti nella cartella specificata.
    for filename in os.listdir(audio_directory_path):
        # Filtra solo i file audio con estensione .mp3 o .wav (case-insensitive).
        if filename.lower().endswith((".mp3", ".wav")):
            audio_file_path = os.path.join(audio_directory_path, filename)
            print(f"\n\n--- INIZIO ELABORAZIONE FILE: {filename} ---")

            # Carica l'intero audio dal file.
            # librosa.load è robusto per vari formati e può rilevare il sample rate.
            print(f"Caricamento dell'intero audio: {audio_file_path}")
            full_audio_data, full_samplerate = librosa.load(audio_file_path, sr=None)
            audio_duration = len(full_audio_data) / full_samplerate
            print(f"L'audio completo è lungo: {audio_duration:.2f} secondi.")

            # Calcola la durata di ogni frammento per ottenere esattamente N_FIXED_SEGMENTS.
            print(f"Dividendo l'audio in esattamente {N_FIXED_SEGMENTS} frammenti di durata fissa.")
            
            # Calcola la durata media di ogni frammento.
            segment_duration_seconds = audio_duration / N_FIXED_SEGMENTS
            # Converti la durata in numero di campioni.
            samples_per_segment = int(segment_duration_seconds * full_samplerate)

            song_descriptions = {} # Dizionario per archiviare le descrizioni dei 5 frammenti per la canzone corrente.
            full_song_description_parts = [] # Lista per costruire la descrizione totale della canzone.

            # Il prompt base per l'utente, che verrà combinato con le istruzioni del sistema.
            base_user_instruction = """Descrivi in 4-6 righe il frammento della canzone fornita, includendo i seguenti aspetti:
1.  **Strumenti presenti**
2.  **Velocità/ritmo**
3.  **Timbro**
4.  **Emozioni suscitate**
5.  **Contesto immaginato**
Cerca di essere il più descrittivo possibile pur mantenendo la lunghezza entro il limite.
"""

            # Loop per processare ogni frammento dell'audio.
            for i in range(N_FIXED_SEGMENTS):
                # Calcola i campioni di inizio e fine per il frammento corrente.
                start_sample = i * samples_per_segment
                # L'ultimo frammento si estende fino alla fine dell'audio per coprire eventuali resti.
                end_sample = min((i + 1) * samples_per_segment, len(full_audio_data))
                
                # Estrai il segmento di audio corrente.
                current_segment = full_audio_data[start_sample:end_sample]

                # Prepara il prompt specifico per questo frammento, indicando il suo numero.
                segment_prompt = f"Analizza il frammento {i+1}/{N_FIXED_SEGMENTS} dell'audio. {base_user_instruction}"
                
                # Esegui l'inferenza sul segmento audio corrente.
                response_text = run_audio_inference_on_segment(
                    audio_segment=current_segment,
                    user_prompt=segment_prompt,
                    model=model,
                    processor=processor,
                    segment_samplerate=full_samplerate,
                    max_new_tokens=256 # Numero massimo di token da generare per la risposta di ogni segmento.
                )
                
                # Archivia la descrizione del frammento con la chiave appropriata per il CSV.
                song_descriptions[f'Descr Frammento {i+1}'] = response_text
                # Aggiungi la descrizione del frammento alla lista per la descrizione totale.
                full_song_description_parts.append(f"Frammento {i+1}: {response_text.strip()}")
                
                # Svuota esplicitamente la cache GPU dopo ogni elaborazione di frammento.
                # Questo è cruciale per la gestione della memoria in un ciclo intensivo.
                torch.cuda.empty_cache()
                print(f"Cache CUDA svuotata dopo l'analisi del frammento {i+1} per {filename}.")
            
            # --- Scrittura della riga completa per la canzone nel CSV ---
            csv_row = {'Nome Canzone': filename}
            csv_row.update(song_descriptions) # Aggiunge le descrizioni dei 5 frammenti al dizionario della riga.
            # Concatena tutte le descrizioni dei frammenti per la "Descrizione Totale", separate da due nuove linee.
            csv_row['Descrizione Totale'] = "\n\n".join(full_song_description_parts)

            writer.writerow(csv_row) # Scrive la riga completa della canzone nel file CSV.

            print(f"--- FINE ELABORAZIONE FILE: {filename} ---\n")
            
            # Una pulizia aggiuntiva della cache GPU dopo aver processato un intero file.
            torch.cuda.empty_cache()
            print(f"Cache CUDA svuotata dopo l'elaborazione completa di {filename}.")

# Messaggio finale che indica il completamento del processo e il nome del file CSV.
print(f"\nProcesso di analisi audio per tutti i file della cartella completato. Risultati salvati in '{csv_output_filename}'.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.3/35.3 MB 42.2 MB/s eta 0:00:00:00:0100:01
/bin/bash: -c: line 1: unexpected EOF while looking for matching ``'
/bin/bash: -c: line 2: syntax error: unexpected end of file


2025-06-12 12:06:06.901628: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749729967.353893      75 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749729967.468525      75 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


*** CONFERMA GPU: PyTorch rileva la GPU 0! Nome: Tesla T4, Memoria Totale: 14.74 GiB
*** CONFERMA GPU: PyTorch rileva la GPU 1! Nome: Tesla T4, Memoria Totale: 14.74 GiB
Caricamento del modello Qwen2-Audio-7B-Instruct...


preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/638k [00:00<?, ?B/s]

...

Modello e processore Qwen2-Audio caricati con successo.
Cache CUDA svuotata dopo il caricamento del modello.

I risultati dell'analisi verranno salvati in: analisi_audio_per_canzone.csv


--- INIZIO ELABORAZIONE FILE: 000207.mp3 ---
Caricamento dell'intero audio: /kaggle/input/le-musiche/PiccoloSubSetAudio/000207.mp3
L'audio completo è lungo: 29.98 secondi.
Dividendo l'audio in esattamente 5 frammenti di durata fissa.

--- Inizio inferenza su segmento audio (lunghezza: 6.00s) ---
Ricasellamento del segmento audio da 44100Hz a 16000Hz.
Preparazione degli input per il modello Qwen2-Audio...


/tmp/ipykernel_75/2761736566.py:112: FutureWarning: `audios` is deprecated and will be removed in version 4.54.0 for `Qwen2AudioProcessor.__call__`. Use `audio` instead.
  inputs = processor(text=text_input_formatted, audios=audios_np_arrays, sampling_rate=final_samplerate, return_tensors="pt", padding=True)


Lunghezza degli input_ids dopo tokenizzazione: 408
Input pronti per la generazione.
Generazione della risposta del modello... Potrebbe richiedere tempo.


/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:2347: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(



--- TESTO GENERATO DAL MODELLO ---

Nel primo minuto e mezzo della canzone si sentono due pianoforti che suonano insieme. La loro melodia è lenta e abbastanza triste da evocare emozioni profonde. I toni sono soft e spenti, creando un senso di solitudine e introspezione. Il contesto immaginato potrebbe essere quello di una scena drammatica o un momento di riflessione in un film.
--- Fine inferenza su segmento audio ---

Cache CUDA svuotata dopo l'analisi del frammento 1 per 000207.mp3.

--- Inizio inferenza su segmento audio (lunghezza: 6.00s) ---
Ricasellamento del segmento audio da 44100Hz a 16000Hz.
Preparazione degli input per il modello Qwen2-Audio...
Lunghezza degli input_ids dopo tokenizzazione: 408
Input pronti per la generazione.
Generazione della risposta del modello... Potrebbe richiedere tempo.

--- TESTO GENERATO DAL MODELLO ---

Nel secondo segmento della canzone si sente un pianoforte che suona una serie di note in modo rapido, con un tono leggermente cupo e un po' spent